In [1]:
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split

from lib.Utility import exportExcelWithTimeStamp

from datetime import date, timedelta
import yfinance as yf #Alternative package if webreader does not work: pip install yfinance
import numpy as np # Fundamental package for scientific computing with Python
import joblib
import plotly.express as px
from plotly import graph_objects as go
# Train the model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

In [2]:
# data = pd.read_excel('serieA-2023-24.xlsx')
# data1920 = pd.read_excel('data/engdata/19-20.xlsx', index_col='Date')
# data2021 = pd.read_excel('data/engdata/20-21.xlsx', index_col='Date')
# data2122 = pd.read_excel('data/engdata/21-22.xlsx', index_col='Date')
# data2223 = pd.read_excel('data/engdata/22-23.xlsx', index_col='Date')
# data2324 = pd.read_excel('data/engdata/23-24.xlsx', index_col='Date')

data1920 = pd.read_excel('data/engdata/PremierLeague/19-20-E.xlsx', index_col='Date')
data2021 = pd.read_excel('data/engdata/PremierLeague/20-21-E.xlsx', index_col='Date')
data2122 = pd.read_excel('data/engdata/PremierLeague/21-22-E.xlsx', index_col='Date')
data2223 = pd.read_excel('data/engdata/PremierLeague/22-23-E.xlsx', index_col='Date')
data2324 = pd.read_excel('data/engdata/PremierLeague/23-24-E.xlsx', index_col='Date')

data = data1920

data = pd.concat([data, data2021, data2122, data2223, data2324])
print(data)
# data['Cluster'] = np.where(data['FTR'] == 'H', 1, np.where(data['FTR'] == 'A', 2, 0))

data.tail()

                        Div      Time          HomeTeam          AwayTeam  \
Date                                                                        
1970-01-01 01:00:43.686  E0  20:00:00         Liverpool           Norwich   
1970-01-01 01:00:43.687  E0  12:30:00          West Ham          Man City   
1970-01-01 01:00:43.687  E0  15:00:00       Bournemouth  Sheffield United   
1970-01-01 01:00:43.687  E0  15:00:00           Burnley       Southampton   
1970-01-01 01:00:43.687  E0  15:00:00    Crystal Palace           Everton   
...                      ..       ...               ...               ...   
1970-01-01 01:00:45.431  E0  16:00:00    Crystal Palace       Aston Villa   
1970-01-01 01:00:45.431  E0  16:00:00         Liverpool            Wolves   
1970-01-01 01:00:45.431  E0  16:00:00             Luton            Fulham   
1970-01-01 01:00:45.431  E0  16:00:00          Man City          West Ham   
1970-01-01 01:00:45.431  E0  16:00:00  Sheffield United         Tottenham   

,Div,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,MaxCAHA,AvgCAHH,AvgCAHA,HomeGoalsCumulative,AwayGoalsCumulative,HomePointsCumulative,AwayPointsCumulative,HomeGoalsConcededCumulative,AwayGoalsConcededCumulative,MatchGoal
Date,,,,,,,,,,,,,,,,,,,,,
1970-01-01 01:00:45.431,E0,16:00:00,Crystal Palace,Aston Villa,5,0,H,2,0,H,...,2.21,1.81,2.05,52,76,46,68,58,56,5
1970-01-01 01:00:45.431,E0,16:00:00,Liverpool,Wolves,2,0,H,2,0,H,...,1.89,2.04,1.82,84,50,79,46,41,63,2
1970-01-01 01:00:45.431,E0,16:00:00,Luton,Fulham,2,4,A,1,2,A,...,1.94,1.96,1.91,50,51,26,44,81,59,6
1970-01-01 01:00:45.431,E0,16:00:00,Man City,West Ham,3,1,H,2,1,H,...,1.99,1.96,1.91,93,59,88,52,33,71,4
1970-01-01 01:00:45.431,E0,16:00:00,Sheffield United,Tottenham,0,3,A,0,1,A,...,1.89,2.01,1.86,35,71,16,63,101,61,3


In [3]:
resultFileName = 'data/mergedDataPremierLeague.xlsx'
data.to_excel(resultFileName)